# pydlem.prep Package Example 1.1
This notebook provides a complete example of acquiring meterology data and combining it with reservoir data to produce a formatted input data file for running the pydlem model.

First import necessary packages/modules

In [1]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
import shapely


pydlem_dir = r'C:\Users\CND905\Downloaded_Programs\pydlem'
os.chdir(pydlem_dir)

from prep.datafile import CreateInputFile
from prep.lakegeom import calc_fetch_length
from prep.metdata import get_gridmet_for_polygons
from prep.datafile import check_format


# # These are in metdata
# from prep.utils import get_gridmet_cells
# from config import GRIDMET_PARAMS
# from chmdata.thredds import GridMet
# from chmdata.thredds import BBox

# # These need to be added
# import GRIDtools as gt
# import rasterio as rio



Initializing pydlem.prep module.


## Static Reservoir Inputs
For this example, we will load the geometry data and extract GridMET meterology for the lakes of interest. In this case, a shapefile is used but this could be extracted from a database or any other method supported by Geopandas.

The 'Permanent_' column was used to index in this example and cannot include character values. In this example, the polygons are loaded witha Z coordinate which needs to be removed.   

In [2]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\pydlem\examples\example_data\HWWY_reservoirs_5070.shp')
exres = gpd.read_file(exres_pth)
# this file is in crs 5070, need it to be 4326 for getting GridMET
exres = exres.to_crs(4326)
exres.geometry = shapely.force_2d(exres.geometry) # Polygon need to be 2D
exres

,Permanent_,FDate,Resolution,GNIS_ID,GNIS_Name,AreaSqKm,Elevation,ReachCode,FType,FCode,Visibility,Shape_Leng,Shape_Area,NHDPlusID,VPUID,geometry
0,71772797,2003-02-20,2,01600849,Last Chance Reservoir,0.051000,0.0000,10090101001988,390,39004,0,0.010722,0.000006,2.300250e+13,1009,"POLYGON ((-107.20213 44.54076, -107.20223 44.5..."
1,71772129,2016-11-03,2,01606753,Colorado Colony Ditch Company Reservoir Number 2,1.257994,0.0000,10090101001985,390,39004,0,0.080258,0.000142,2.300250e+13,1009,"POLYGON ((-107.21702 44.54794, -107.21707 44.5..."
2,71772161,2003-02-20,2,01600017,Granger Reservoir,0.028000,0.0000,10090101001980,390,39004,0,0.008714,0.000003,2.300250e+13,1009,"POLYGON ((-107.2024 44.56734, -107.2023 44.567..."
3,71759682,2003-02-20,2,01606767,Dome Lake Number 1,0.204000,0.0000,10090101001957,390,39004,0,0.022006,0.000023,2.300250e+13,1009,"POLYGON ((-107.31638 44.60378, -107.3165 44.60..."
4,71772927,2003-02-20,2,01608139,Willits Reservoir,0.026000,0.0000,10090101001989,390,39004,0,0.007028,0.000003,2.300250e+13,1009,"POLYGON ((-107.19553 44.53306, -107.19525 44.5..."
5,71759808,2003-02-20,2,01587705,Dome Lake Reservoir,0.357000,0.0000,10090101001969,390,39004,0,0.031711,0.000040,2.300250e+13,1009,"POLYGON ((-107.30002 44.58735, -107.30028 44.5..."
6,71818121,2003-02-20,2,01604632,Sawmill Reservoir,0.265000,0.0000,10090101001947,390,39004,0,0.030454,0.000030,2.300250e+13,1009,"POLYGON ((-107.30803 44.62497, -107.30803 44.6..."
7,71759956,2003-02-20,2,01596266,Weston Reservoir,0.138000,0.0000,10090101001976,390,39004,0,0.019932,0.000016,2.300250e+13,1009,"POLYGON ((-107.26949 44.56849, -107.26952 44.5..."
8,120031076,2012-11-26,2,01599036,Cross Creek Reservoir,0.223523,2745.6384,10090101001994,390,39009,0,0.022913,0.000025,2.300250e+13,1009,"POLYGON ((-107.20593 44.5041, -107.20575 44.50..."
9,71772901,2016-11-03,2,01606391,Big Horn Reservoir,0.669102,0.0000,10090101001993,390,39004,0,0.042110,0.000076,2.300250e+13,1009,"POLYGON ((-107.20568 44.52121, -107.20608 44.5..."


## Get Meterology Inputs
The get_gridmet_fpr_polygons function was used to make dataset of all inputs needed for the DLEM model. The xvec and exactextract packages are used here.

In [3]:
exres_met = get_gridmet_for_polygons(exres, 'Permanent_', '2024-01-01', '2024-12-31')
exres_met

c:\Users\CND905\.conda\envs\pydlem\lib\site-packages\exactextract\exact_extract.py:330: RuntimeWarning: Spatial reference system of input features does not exactly match raster.
  warnings.warn(
c:\Users\CND905\.conda\envs\pydlem\lib\site-packages\exactextract\exact_extract.py:330: RuntimeWarning: Spatial reference system of input features does not exactly match raster.
  warnings.warn(


<xarray.Dataset> Size: 229kB
Dimensions:        (location: 11, time: 366)
Coordinates:
    lat            (location) float64 88B 44.54 44.56 44.57 ... 44.5 44.53 44.55
    lon            (location) float64 88B -107.2 -107.2 -107.2 ... -107.2 -107.2
    elev           (location) float64 88B 2.638e+03 2.517e+03 ... 2.618e+03
    area           (location) float64 88B 0.05123 1.258 ... 0.6691 0.1195
  * location       (location) <U9 396B '71772797' '71772129' ... '71772619'
  * time           (time) datetime64[ns] 3kB 2024-01-01 ... 2024-12-31
Data variables:
    max_temp       (time, location) float64 32kB 273.4 273.7 ... 265.6 266.3
    min_temp       (time, location) float64 32kB 262.7 263.0 ... 255.5 258.1
    precip_volume  (time, location) float64 32kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    solrad         (time, location) float64 32kB 76.3 76.2 76.1 ... 50.65 50.1
    wind_dir       (time, location) float64 32kB 308.0 308.0 ... 305.0 305.0
    wind_vel       (time, location) float64 32kB 3.6 3.6 3.6 ... 4.2 4.05 3.9
    vpd            (time, location) float64 32kB 0.2 0.21 0.22 ... 0.075 0.09
Attributes:
    featureType:  timeSeries

Since the precipitation variable is returned as a volume (m^3), we need to convert it back to a length (mm) which is what the model requires.

In [5]:
precip = exres_met.precip_volume.values / (np.tile(exres_met.area.values, (len(exres_met.time), 1)) * (1000**2)) # divide precip volume(m^3) by lake area (converted from km^2 to m^2). Returns precip (m)
precip = precip * 1000 # convert from m to mm
precip_xda = xr.DataArray(precip, coords=exres_met.precip_volume.coords, attrs={'standard_name': 'Precipitation', 'units': 'mm'})
precip_xda.name = 'precip'

exres_met = xr.merge([exres_met.drop_vars('precip_volume'), precip_xda]) # replace precip_volume with precip variable

We can save the xarray.Dataset at this point to show that .nc files can also be loaded in to run the model instead. 

In [7]:
pydlem_dir = Path(r'C:\Users\CND905\Downloaded_Programs\pydlem\examples\example_data')
os.chdir(pydlem_dir)
exres_met.to_netcdf(pydlem_dir / 'prep_Example1.1_met.nc')

## Calculate Lake Area, Depth and Fetch
Lake Area and Depth are required inputs to create the datafile, as pydlem needs these to run the computations. So first, we will create these and then load inputs from our previos meteorlogy netcdf file. The easiest way to start with pydlem is to assume static values for reservoir variables 'Depth' and 'Surface Area'.

In [8]:
# surface area from NHD
lareas = np.array([exres[exres['Permanent_'] == x]['AreaSqKm'].iloc[0] for x in exres_met.location.values.astype(str)])
lareas_arr = np.tile(lareas, (len(exres_met.time + 1), 1))
# in this case we will make it an xarray DataArray
Lake_Area = xr.DataArray(lareas_arr, coords=exres_met.coords, attrs={'standard_name': 'Lake Surface Area', 'units': 'km^2'})
Lake_Area.name = 'LakeArea'

# Next, to get depth we need volume. You can estimate the volume or assume an average value for depth. In this case, the reservoirs have max capacity 
# data that are manually listed here and used to calcuate depth.

# These are in cubic kilometers
caps = np.array([0.000259, 0.0128, 0.00018, 0.0042, 0.000099, 0.00186, 0.00157, 0.00046, 0.00098, 0.0057, 0.00069])
caps_arr = np.tile(caps, (len(exres_met.time + 1), 1))
ldpth = (caps_arr / lareas_arr) * 1000 # convert to meters of depth
Lake_Depth = xr.DataArray(ldpth, coords=exres_met.coords, attrs={'standard_name': 'Lake Depth', 'units': 'm'})
Lake_Depth.name = 'LakeDepth'

Now we load our saved netcdf from file into xarray and create the input data file.

In [ ]:
met_ff = xr.load_dataset(r'C:\Users\CND905\Downloaded_Programs\pydlem\examples\example_data\prep_Example1.1_met.nc')
nw_Idat = CreateInputFile(geoms=None, lake_area=Lake_Area, lake_depth=Lake_Depth, met_data=met_ff, met_source='from_file')

We still are missing some required variables. 'ftch_len' is the FETCH for the individual lakes. This is an important term in the dlem model that accounts for wind blowing across the lakes surface. The lakegeom.py module within the pydlem.prep package has a method to calculate fetch length needed to run the model.

We also need the mean_temp but this is calculated in the model.py code so the max and min temperature parameters are only required. 

In [ ]:
# calculating the fetch length
# fetch is very dynamic, it depends on lake area, wind direction, and surface area geometry which are always flucuating
# thus, the fetch length function calculates only one fetch length at a time, so input arrays need to be looped over their 2 dimensions
# (time, location) calculating fetch length for all values

# assign necessary variables and complete any conversions
rows = nw_Idat.data.wind_dir.shape[0]
cols = nw_Idat.data.wind_dir.shape[1]
wnd_d = nw_Idat.data.wind_dir.values
lk_A = nw_Idat.data.LakeArea.values * 1000.0**2  # convert to meters
# Loop through data arrays of wind direction for each timestamp(x) and location(y)
resvs = exres.to_crs(exres.estimate_utm_crs())
xs = []
for x in range(0, rows):
    ys = []
    for y in range(0, cols):
        gmtry = resvs.loc[resvs['Permanent_'] == nw_Idat.data.location[0].values.astype(str)].geometry
        wnd = wnd_d[x, y]
        A = lk_A[x, y]
        ftchL = calc_fetch_length(gmtry, wnd, A)
        ys.append(ftchL)
    xs.append(ys)
# create xarray DataArray from the resulting ndarray
flen = np.array(xs)
flen = xr.DataArray(flen, coords=nw_Idat.data.coords, attrs={'standard_name': 'Wind Fetch Length', 'units': 'm'})
# Add to the dataset
nw_Idat.add_variable(flen, 'ftch_len')

Nearly, all variables required by the model are added. lrad can be estimated in the model but is not required.  

In [ ]:
check_format(nw_Idat.data)

... And here is the final dataset.

In [ ]:
nw_Idat.data

We now have a successfully built input file for pydlem. We can save it as Example 1.1 for use in the model running example notebook.

In [14]:
nw_Idat.save_datafile('Example1.1_static_inputs.nc')
